In [18]:
import pandas
import requests
import xml.etree.ElementTree as ET

df = None
resumptionToken = None

while True:
    # Adresa API - jde otevřít normálně v prohlížeči
    if not resumptionToken:
        response = requests.get('https://api.aiscr.cz/dapro/oai?verb=ListRecords&metadataPrefix=oai_amcr&set=akce')
    else:
        response = requests.get(f'https://api.aiscr.cz/dapro/oai?verb=ListRecords&resumptionToken={resumptionToken}')
    # Úprava kódování
    response.encoding = "utf-8"

    try:
        df_new = pandas.read_xml(response.text, xpath=".//akce", parser="etree")
    except ValueError as err:
        # Uložení dat, pokud se nepovedlo parsování
        with open('error_data.xml', 'w', encoding="utf-8") as f:
            print(response.text, file=f)
    # Metoda set_index nastaví index, který v pandas funguje podobně jako v databázích. Nastavíme jako index sloupec ident_cely
    df_new = df_new.set_index("ident_cely")
    # Hlavička - podívám se na prvních pět řádku, jestli se data načetla v pořádku
    df_new.head()
    if df is not None:
        df = pandas.concat([df, df_new])
    else:
        df = df_new

    # Uložení dat do souboru - pouze pro kontrolu
    with open('list_records.xml', 'w', encoding="utf-8") as f:
        print(response.text, file=f)

    ns = {'oai': 'http://www.openarchives.org/OAI/2.0/'}

    root = ET.fromstring(response.text)
    resumptionToken = root.find('.//oai:resumptionToken', ns).text
    try:
        print(root.find('.//oai:resumptionToken', ns).attrib['cursor'])
    except KeyError as err:
        # Kurzor chybí u poslední dávky
        print("Missing cursor")
        break
df.to_csv("data.csv")    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


Toto je nejsložitější část. Data jsou ve formátu XML, který je založený na používání párových značek. Například hodnota `vedouci_akce` je vložena do párových značek takto: `<vedouci_akce>Kos, Petr</vedouci_akce>`. Celá akce je pak uložená v párové značce `<akce></akce>`. My chceme načíst všechny párové značky `akce`, což za nás provede parametr `xpath=".//akce`. Použitý je vyhledávají jazyk XPath, ale to řešit nemusíte, s ním už jinde asi pracovat nebudeme.

Data jsou načtena do tabulky modulu `pandas`. Práci s modulem `pandas` budete probírat v rámci předmětu Python Data. Funguje podobně jako SQL, takže můžete vybírat konkrétní sloupečky, filtrovat řádky atd.

![structure.PNG](assets/structure.PNG)

,typ,stav,je_nz,okres,katastr,vedouci_akce,organizace,uzivatelske_oznaceni,specifikace_data,datum_zahajeni,...,child_dok_jednotka,datum_podani_nz,odpovedny_pracovnik_podani_nz,datum_archivace,odpovedny_pracovnik_archivace,vazba_projekt_akce,child_dokument,dalsi_katastry,vedouci_akce_ostatni,organizace_ostatni
ident_cely,,,,,,,,,,,,,,,,,,,,,
M-9102504A,N,5,0,BŘECLAV,HUSTOPEČE U BRNA,"Kos, Petr",ÚAPP Brno,143/2004,přesně,0200-09-08,...,M-9102504A-D01,None,None,None,None,NaN,None,None,None,None
M-9105056A,N,8,1,OPAVA,SLAVKOV U OPAVY,"Malík, Petr",NPÚ ú.o.p. Ostrava,062/2011,přesně,0201-08-09,...,M-9105056A-D01,2023-01-18,"Anonym, ARÚB",2023-01-18,"Anonym, ARÚB",NaN,None,None,None,None
M-9106217A,N,8,1,OPAVA,VÍTKOV,"Krasnokutská, Tereza",NPÚ ú.o.p. Ostrava,098/2016,přesně,0201-09-19,...,M-9106217A-D01,2023-02-01,"Anonym, ARÚB",2023-02-01,"Anonym, ARÚB",NaN,None,None,None,None
M-9106246A,N,8,1,OPAVA,ŠTĚPÁNKOVICE,"Krasnokutská, Tereza",NPÚ ú.o.p. Ostrava,065/2016,přesně,0206-06-19,...,M-9106246A-D01,2023-01-25,"Anonym, ARÚB",2023-01-25,"Anonym, ARÚB",NaN,None,None,None,None
C-201663968A,R,8,0,PLZEŇ-SEVER,KRALOVICE U RAKOVNÍKA,"Stráník, Daniel",Muzeum Mariánská Týnice,"AR 8/2016 - Kralovice, Za Rájem, výstavba RD",přesně,0216-05-01,...,C-201663968A-D01,2022-02-21,"Stráník, Daniel",2022-07-14,"Švejcar, Ondřej",NaN,C-TX-202200118,None,None,None
